# Charged Gel

Amphiphilic micelles gelification by charging

In [ ]:
from lammps import PyLammps

In [ ]:
L = PyLammps()

#### SET SIMULATION CONDITIONS

In [ ]:
L.command("units         lj")
L.command("atom_style    full")
L.command("bond_style harmonic")
L.command("pair_style lj/expand 2.5")
L.command("kspace_style pppm 1.0e-4")

L.command("dimension     3")
L.command("boundary p p p")
L.command("neighbor 0.8 bin")

In [ ]:
L.command("read_data neutralMicelles.data")
L.command("group charged type 2")

####  VARIABLES SETUP

In [ ]:
L.command("variable Temp equal 1.0")
L.command("variable dt equal 0.005")
L.command("variable Tdamping equal 100*${dt}")

L.command("thermo  50")
L.command("thermo_style    custom step temp ebond epair ke etotal enthalpy vol press")

L.command("velocity     all create ${Temp} 2398481")
L.command("timestep     ${dt}")

#### ARMS CHARGING

In [ ]:
L.command("set type 2 charge -0.001")
L.command("variable rampCharge equal ramp(-0.001,-1.0)")
L.command("fix charging charged adapt 10 atom charge v_rampCharge")

L.command("pair_style hybrid/overlay lj/cut 2.5 coul/long 10.0")
L.command("pair_coeff * * lj/cut 10.0 1.0 2.5")
L.command("pair_coeff 1 2 lj/cut 1.0 1.0 1.12")
L.command("pair_coeff 2 * lj/cut 1.0 1.0 1.12")
L.command("pair_coeff * * coul/long")

L.command("kspace_style pppm 1.0e-4")

L.command("fix nve1 all nve")
L.command("fix solvent all langevin 1.0 1.0 100.0 8434572")

L.command("dump dumpCharges all custom 20 Conf.dat id type x y z vx vy vz")
L.command("dump_modify dumpCharges sort id")

L.command("reset_timestep 0")
L.command("run 1000")

In [ ]:
import nglview as nv
import MDAnalysis as mda

In [ ]:
u = mda.Universe('Conf.dat',format='LAMMPSDUMP')
atoms = u.atoms

In [ ]:
view = nv.show_mdanalysis(atoms)

In [ ]:
view